# Riddler Express May 6, 2021

From https://fivethirtyeight.com/features/can-you-systematically-solve-a-friday-crossword/

You’re playing a game of cornhole with your friends, and it’s your turn to toss the four bean bags. For every bean bag you toss onto your opponents’ board, you get 1 point. For every bag that goes through the hole on their board, you get 3 points. And for any bags that don’t land on the board or through the hole, you get 0 points.

Your opponents had a terrible round, missing the board with all their throws. Meanwhile, your team currently has 18 points — just 3 points away from victory at 21. You’re also playing with a special house rule: To win, you must score exactly 21 points, without going over.

Based on your history, you know there are three kinds of throws you can make:

 - An aggressive throw, which has a 40 percent chance of going in the hole, a 30 percent chance of landing on the board and a 30 percent chance of missing the board and hole.
 - A conservative throw, which has a 10 percent chance of going in the hole, an 80 percent chance of landing on the board and a 10 percent chance of missing the board and hole.
 - A wasted throw, which has a 100 percent chance of missing the board and hole.

For each bean bag, you can choose any of these three throws. Your goal is to maximize your chances of scoring exactly 3 points with your four tosses. What is the probability that your team will finish the round with exactly 21 points and declare victory?

## Solution

We build solutions to the problem with backwards induction: we will find optimal strategies for when you have 1, 2, or 3 points to make up with just 1 toss, or 2 tosses, etc., and build up to 4. 

For notation, we simplify things by considering 3 the target number of points (since 3 is the deficit you must meet in the problem) and speak of you having only 0, 1, or 2 when you are shy of that.

Thus, the trivial case of having 0 tosses remaining yields the following probabilities of victory: 

| # Tosses Left | 0 points | 1 point | 2 points |
| --- | --- | --- | --- |
| 0 | 0 | 0 | 0 |

When there is 1 toss remaining? Needing exactly 2 points still leaves the player in a hopeless position. Needing exactly three mandates an Aggressive throw--.4 chance > .1. And needing exactly 1 more point mandates a conservative throw, .8 change of victory > .3.

| # Tosses Left | 0 points | 1 point | 2 points |
| --- | --- | --- | --- |
| 0 | 0 | 0 | 0 |
| 1 | .40 (A) | 0 | .80 (C) |

Things get slightly more complicated with 2 remaining tosses: with 0 points, an Aggressive throw's chance of success is 0.4 x 1 + 0.3 x (victory chance of 1 toss, 1 point) + 0.3 x (victory chance of 1 toss, 0 points) = .4 + .12 = 0.52; a Conservative throw's chance of success is 0.8 x (victory chance of 1 toss, 1 point) + .1 x (victory chance of 1 toss, 0 points) + .1 x (victory) = 0.1.

With 1 point, a Conservative toss's chance of victory is 0.8 x (vc of 1 toss, 2 points) + 0.1 x (vc of 1 toss, 1 point) + 0.1 x (bust, at 4 points) = 0.8 x 0.8 = 0.64. It is clear that an Aggressive throw's .40 chance of ending the game in a loss means that its chance of victory cannot surpass 0.64.

With 2 points, a Conservative toss has an 80% chance of immediate victory, a 10% chance of an immediate loss, and a 10% chance of (vc of 1 toss, 2 points = 0.8) = 0.80 + 01 x 0.8 = 0.88.

| # Tosses Left | 0 points | 1 point | 2 points |
| --- | --- | --- | --- |
| 0 | 0 | 0 | 0 |
| 1 | .40 (A) | 0 | .80 (C) |
| 2 | .52 (A) | 0.64 (C) | .88 (C) |

We now turn to Python to automate the above calculations to extend our table of win probabilities given the throwing mechanics, a certain number of points left, and a certain number of tosses left. 

In [1]:
import pandas as pd
d = {'0 points': [0, .4, .52], '1 point': [0, 0, .64], '2 points': [0, .8, .88]}
df = pd.DataFrame(data=d)
df

,0 points,1 point,2 points
0,0.00,0.00,0.00
1,0.40,0.00,0.80
2,0.52,0.64,0.88


In [2]:
def strategize(throws, points):
    c_chance = 0
    if points==2:
        c_chance += 0.8 * 1
    else:
        c_chance += 0.8 * (df.iloc[throws-1, points+1])
    try:
        c_chance += 0.1 * (df.iloc[throws-1, points+3])
    except IndexError:
        pass
    c_chance += 0.1 * (df.iloc[throws-1, points])

    a_chance = 0
    if points==0:
        a_chance += 0.4 * 1
        a_chance += 0.3 * (df.iloc[throws-1, points+1])
        a_chance += 0.3 * (df.iloc[throws-1, points])
    if c_chance > a_chance:
        return c_chance, "Conservative"
    else:
        return a_chance, "Aggressive"

In [3]:
def add_row():
    global df
    strats = []
    throws = df.shape[0]
    for points in range(3):
        strats.append(strategize(throws, points))
    chances = [x[0] for x in strats]
    df2 = pd.DataFrame([chances], columns=df.columns)
    df=df.append(df2, ignore_index = True)
    df = df.round(4)
    display(df)
    return strats

In [4]:
for i in range(6):
    print(add_row())

,0 points,1 point,2 points
0,0.000,0.000,0.000
1,0.400,0.000,0.800
2,0.520,0.640,0.880
3,0.748,0.768,0.888


[(0.7480000000000001, 'Aggressive'), (0.768, 'Conservative'), (0.888, 'Conservative')]


,0 points,1 point,2 points
0,0.0000,0.0000,0.0000
1,0.4000,0.0000,0.8000
2,0.5200,0.6400,0.8800
3,0.7480,0.7680,0.8880
4,0.8548,0.7872,0.8888


[(0.8548, 'Aggressive'), (0.7872, 'Conservative'), (0.8888, 'Conservative')]


,0 points,1 point,2 points
0,0.0000,0.0000,0.0000
1,0.4000,0.0000,0.8000
2,0.5200,0.6400,0.8800
3,0.7480,0.7680,0.8880
4,0.8548,0.7872,0.8888
5,0.8926,0.7898,0.8889


...

[(0.90472, 'Aggressive'), (0.7901, 'Conservative'), (0.8888900000000001, 'Conservative')]


,0 points,1 point,2 points
0,0.0000,0.0000,0.0000
1,0.4000,0.0000,0.8000
2,0.5200,0.6400,0.8800
3,0.7480,0.7680,0.8880
4,0.8548,0.7872,0.8888
5,0.8926,0.7898,0.8889
6,0.9047,0.7901,0.8889
7,0.9084,0.7901,0.8889


[(0.9084399999999999, 'Aggressive'), (0.7901300000000001, 'Conservative'), (0.8888900000000001, 'Conservative')]


,0 points,1 point,2 points
0,0.0000,0.0000,0.0000
1,0.4000,0.0000,0.8000
2,0.5200,0.6400,0.8800
3,0.7480,0.7680,0.8880
4,0.8548,0.7872,0.8888
5,0.8926,0.7898,0.8889
6,0.9047,0.7901,0.8889
7,0.9084,0.7901,0.8889
8,0.9096,0.7901,0.8889


[(0.90955, 'Aggressive'), (0.7901300000000001, 'Conservative'), (0.8888900000000001, 'Conservative')]


We note that, needing *exactly* three more points, the strategy is always Aggressive for 3 points needed, Conservative for 1 or 2. Thus, the table displays the probabilities that we will win, given whichever square we find ourselves in. 